In [30]:
from sage.crypto.boolean_function import BooleanFunction
import numpy as np
import pandas as pd
from multiprocess import Pool


In [31]:
state_power = 6

B = BooleanPolynomialRing(state_power, 'x', order = 'lex')
B.inject_variables()


Defining x0, x1, x2, x3, x4, x5


In [32]:
with open('./combining-function.txt') as combining_function_file:
	combining_function_string = combining_function_file\
		.read()\
		.replace('_{', '')\
		.replace('} x', ' * x')\
		.replace('} +', ' +')

	combining_function = eval(combining_function_string)

combining_function


x0*x4*x5 + x1*x4*x5 + x2*x4*x5 + x2*x5 + x3*x4*x5 + x3*x5 + x4*x5 + x4 + x5 + 1

In [33]:
with open('./validated-gamma.txt') as validated_gamma_file:
	validated_gamma = validated_gamma_file.read()
	validated_gamma = [int(bit) for bit in validated_gamma]

validated_gamma[:10]


[1, 0, 0, 1, 0, 0, 1, 0, 0, 1]

In [34]:
def poly_to_int(poly):
	return int(''.join([str(i) for i in poly.list()[:-1]]), 2)

polynomials = [
	x^10 + x^3 + 1,
	x^10 + x^7 + 1,
	x^10 + x^4 + x^3 + x + 1,
	x^10 + x^8 + x^3 + x^2 + 1,
	x^10 + x^8 + x^4 + x^3 + 1,
	x^10 + x^8 + x^5 + x + 1,
]

polynomials_int = [poly_to_int(poly) for poly in polynomials]

polynomials_int


[576, 516, 864, 706, 610, 786]

In [35]:
walsh_spectre = list(BooleanFunction(combining_function).walsh_hadamard_transform())

print(walsh_spectre)


[0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, -32, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, 0, -16, 0, 0, 0, -32, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, -16, 0, 0, 0]


In [36]:
def int_to_binary(number):
	return [(number >> j) & 1 for j in range(state_power)]

In [37]:
stringified_analogs = []
analogs = {}
analogs_with_walsh = {}

for index in range(len(walsh_spectre)):
	coefficient = walsh_spectre[index]
	analog_monomials = []

	index_binary = int_to_binary(index)

	for i, a_i in enumerate(index_binary):
		if a_i == 1:
			analog_monomials.append(f'x{i}')
	
	if (sign(coefficient) == -1):
		analog_monomials.append('1')

	if len(analog_monomials) > 0:
		stringified_analog = ' + '.join(analog_monomials)

		stringified_analogs.append(stringified_analog)
		analogs[stringified_analog] = BooleanFunction(eval(stringified_analog))
		analogs_with_walsh[stringified_analog] = coefficient

stringified_analogs


['x0',
 'x1',
 'x0 + x1',
 'x2',
 'x0 + x2',
 'x1 + x2',
 'x0 + x1 + x2',
 'x3',
 'x0 + x3',
 'x1 + x3',
 'x0 + x1 + x3',
 'x2 + x3',
 'x0 + x2 + x3',
 'x1 + x2 + x3',
 'x0 + x1 + x2 + x3',
 'x4 + 1',
 'x0 + x4',
 'x1 + x4',
 'x0 + x1 + x4 + 1',
 'x2 + x4',
 'x0 + x2 + x4',
 'x1 + x2 + x4',
 'x0 + x1 + x2 + x4',
 'x3 + x4',
 'x0 + x3 + x4',
 'x1 + x3 + x4',
 'x0 + x1 + x3 + x4',
 'x2 + x3 + x4',
 'x0 + x2 + x3 + x4',
 'x1 + x2 + x3 + x4',
 'x0 + x1 + x2 + x3 + x4',
 'x5',
 'x0 + x5',
 'x1 + x5',
 'x0 + x1 + x5 + 1',
 'x2 + x5',
 'x0 + x2 + x5',
 'x1 + x2 + x5',
 'x0 + x1 + x2 + x5',
 'x3 + x5',
 'x0 + x3 + x5',
 'x1 + x3 + x5',
 'x0 + x1 + x3 + x5',
 'x2 + x3 + x5 + 1',
 'x0 + x2 + x3 + x5',
 'x1 + x2 + x3 + x5',
 'x0 + x1 + x2 + x3 + x5',
 'x4 + x5 + 1',
 'x0 + x4 + x5',
 'x1 + x4 + x5',
 'x0 + x1 + x4 + x5',
 'x2 + x4 + x5',
 'x0 + x2 + x4 + x5',
 'x1 + x2 + x4 + x5',
 'x0 + x1 + x2 + x4 + x5',
 'x3 + x4 + x5',
 'x0 + x3 + x4 + x5',
 'x1 + x3 + x4 + x5',
 'x0 + x1 + x3 + x4 + x5',
 '

In [38]:
boolean_combining_function = BooleanFunction(combining_function)

probabilities = dict(zip(stringified_analogs, np.zeros(len(stringified_analogs))))

for stringified_analog in stringified_analogs:
	for i in range(1 << state_power):
		if boolean_combining_function(i) == analogs[stringified_analog](i):
			probabilities[stringified_analog] += 1

for analog in probabilities.keys():
	probabilities[analog] /= (1 << state_power)

probabilities = dict(sorted(probabilities.items(), key = lambda x: x[1], reverse = True))

probabilities


{'x4 + 1': 0.75,
 'x4 + x5 + 1': 0.75,
 'x0 + x1': 0.625,
 'x2 + x3': 0.625,
 'x0 + x1 + x4 + 1': 0.625,
 'x2 + x3 + x4': 0.625,
 'x0 + x1 + x5 + 1': 0.625,
 'x2 + x3 + x5 + 1': 0.625,
 'x0 + x1 + x4 + x5': 0.625,
 'x2 + x3 + x4 + x5 + 1': 0.625,
 'x0': 0.5,
 'x1': 0.5,
 'x2': 0.5,
 'x0 + x2': 0.5,
 'x1 + x2': 0.5,
 'x0 + x1 + x2': 0.5,
 'x3': 0.5,
 'x0 + x3': 0.5,
 'x1 + x3': 0.5,
 'x0 + x1 + x3': 0.5,
 'x0 + x2 + x3': 0.5,
 'x1 + x2 + x3': 0.5,
 'x0 + x1 + x2 + x3': 0.5,
 'x0 + x4': 0.5,
 'x1 + x4': 0.5,
 'x2 + x4': 0.5,
 'x0 + x2 + x4': 0.5,
 'x1 + x2 + x4': 0.5,
 'x0 + x1 + x2 + x4': 0.5,
 'x3 + x4': 0.5,
 'x0 + x3 + x4': 0.5,
 'x1 + x3 + x4': 0.5,
 'x0 + x1 + x3 + x4': 0.5,
 'x0 + x2 + x3 + x4': 0.5,
 'x1 + x2 + x3 + x4': 0.5,
 'x0 + x1 + x2 + x3 + x4': 0.5,
 'x5': 0.5,
 'x0 + x5': 0.5,
 'x1 + x5': 0.5,
 'x2 + x5': 0.5,
 'x0 + x2 + x5': 0.5,
 'x1 + x2 + x5': 0.5,
 'x0 + x1 + x2 + x5': 0.5,
 'x3 + x5': 0.5,
 'x0 + x3 + x5': 0.5,
 'x1 + x3 + x5': 0.5,
 'x0 + x1 + x3 + x5': 0.5,
 'x0

In [39]:
best_of_the_best = list(probabilities.keys())[:10]

candidates = [analogs[stringified_analog] for stringified_analog in best_of_the_best]

best_of_the_best


['x4 + 1',
 'x4 + x5 + 1',
 'x0 + x1',
 'x2 + x3',
 'x0 + x1 + x4 + 1',
 'x2 + x3 + x4',
 'x0 + x1 + x5 + 1',
 'x2 + x3 + x5 + 1',
 'x0 + x1 + x4 + x5',
 'x2 + x3 + x4 + x5 + 1']

In [40]:
%%cython

from libc.stdlib cimport malloc, free
cdef int LENGTH = 10

def generate_sequence(int poly, int init_state, int nbits):
	cdef int curr_state = init_state, s, i, j
	cdef int* res_array = <int*>malloc(sizeof(int)*nbits)

	try:
		for i in range(nbits):
			res_array[i] = (curr_state >> (LENGTH - 1)) & 1
			s = 0
			
			for j in range(LENGTH):
				s = s ^ (((curr_state & poly) >> j) & 1)
			
			curr_state = (curr_state << 1) | s
			
		return [x for x in res_array [:nbits]]
	finally:
		free(res_array)


In [41]:
def state_to_int(*params):
	number = 0

	for index, bit in params:
		temp = bit << index

		number |= temp

	return number


In [42]:
brutforce_power = 1 << 10

def get_bit_vector(*params):
	vector = [0 for i in range(6)]

	for index, bit in params:
		vector[index] = bit

	return vector

def get_T(k):
	return int(8 * 0.5 ** -2 * np.log(2 ** (10 * k) * 100))

def get_bit_difference(first_sequence, second_sequence):
	return sum([a != b for a, b in zip(first_sequence, second_sequence)])

def get_sorted_dict(dictionary):
	return list(sorted(dictionary.items(), key = lambda x: x[1]))


In [43]:
def brutforce_for_one_variable(init_state, poly_index, g, T):
	sequence = generate_sequence(polynomials_int[poly_index], init_state, T)
	test_sequence = [g(state_to_int((poly_index, bit))) for bit in sequence]

	return (init_state, get_bit_difference(test_sequence, validated_gamma[:T]))


In [44]:
%%time

g_0 = candidates[0]
T_0 = get_T(2)

print(f'g_0 = {best_of_the_best[0]}')
print(f'')
print(f'T = {T_0}')
print(f'----------')

with Pool() as pool:
	differences_0 = dict(
		pool.map(
			lambda init_state: brutforce_for_one_variable(init_state, 4, g_0, T_0),
			range(brutforce_power)
		)
	)

pd.DataFrame(get_sorted_dict(differences_0)[:10], columns = ['Initial State', 'Hemming Distance'])


g_0 = x4 + 1

T = 590
----------
CPU times: user 42 ms, sys: 70.4 ms, total: 112 ms
Wall time: 157 ms


,Initial State,Hemming Distance
0,51,147
1,373,259
2,63,261
3,521,264
4,86,266
5,175,266
6,781,269
7,934,269
8,23,270
9,850,270


In [45]:
poly_state_4 = 51


In [46]:
def brutforce_for_one_variable_custom(init_state, poly_index_2, g, T, generated_sequence):
	poly_index_1, sequence_1 = generated_sequence

	sequence_2 = generate_sequence(polynomials_int[poly_index_2], init_state, T)

	test_sequence = [
		g(state_to_int((poly_index_1, sequence_1[i]), (poly_index_2, sequence_2[i])))
		for i in range(T)
	]

	return (init_state, get_bit_difference(test_sequence, validated_gamma[:T]))


In [47]:
%%time

g_1 = candidates[1]
T_1 = get_T(2)

print(f'g_1 = {best_of_the_best[1]}')
print(f'')
print(f'T = {T_1}')
print(f'----------')

sequence_poly_4 = generate_sequence(polynomials_int[4], poly_state_4, T_1)

with Pool() as pool:
	differences_1 = dict(
		pool.map(
			lambda init_state: brutforce_for_one_variable_custom(init_state, 5, g_1, T_1, (4, sequence_poly_4)),
			range(brutforce_power)
		)
	)

pd.DataFrame(get_sorted_dict(differences_1)[:10], columns = ['Initial State', 'Hemming Distance'])


g_1 = x4 + x5 + 1

T = 590
----------
CPU times: user 26.5 ms, sys: 76.4 ms, total: 103 ms
Wall time: 166 ms


,Initial State,Hemming Distance
0,0,147
1,951,150
2,279,264
3,16,266
4,41,266
5,328,266
6,451,267
7,799,267
8,397,269
9,904,269


In [48]:
poly_state_5 = 951


In [49]:
def brutforce_for_two_variable(init_state_1, polynomials_indexes, g, T):
	poly_index_1, poly_index_2 = polynomials_indexes

	differences = []

	sequence_1 = generate_sequence(polynomials_int[poly_index_1], init_state_1, T)

	for init_state_2 in range(brutforce_power):
		sequence_2 = generate_sequence(polynomials_int[poly_index_2], init_state_2, T)

		test_sequence = [
			g(state_to_int((poly_index_1, sequence_1[i]), (poly_index_2, sequence_2[i]))) 
			for i in range(T)
		]

		differences.append(((init_state_1, init_state_2), get_bit_difference(test_sequence, validated_gamma[:T])))

	return differences


In [50]:
%%time

g_2 = candidates[2]
T_2 = get_T(3)

print(f'g_2 = {best_of_the_best[2]}')
print(f'')
print(f'T = {T_2}')
print(f'----------')

with Pool() as pool:
	differences_2 = list(
		pool.map(
			lambda init_state_1: brutforce_for_two_variable(init_state_1, (0, 1), g_2, T_2),
			range(brutforce_power)
		)
	)

	differences_2 = dict([difference for sublist in differences_2 for difference in sublist])

pd.DataFrame(get_sorted_dict(differences_2)[:10], columns = ['Initial State', 'Hemming Distance'])


g_2 = x0 + x1

T = 812
----------
CPU times: user 923 ms, sys: 161 ms, total: 1.08 s
Wall time: 2min


,Initial State,Hemming Distance
0,"(781, 328)",331
1,"(603, 100)",335
2,"(637, 581)",341
3,"(755, 548)",341
4,"(193, 4)",344
5,"(759, 686)",344
6,"(986, 178)",344
7,"(98, 42)",345
8,"(697, 120)",345
9,"(873, 769)",345


In [51]:
poly_state_0 = 781
poly_state_1 = 328


In [52]:
%%time

g_3 = candidates[3]
T_3 = get_T(3)

print(f'g_3 = {best_of_the_best[3]}')
print(f'')
print(f'T = {T_3}')
print(f'----------')

with Pool() as pool:
	differences_3 = list(
		pool.map(
			lambda init_state_1: brutforce_for_two_variable(init_state_1, (2, 3), g_3, T_3),
			range(brutforce_power)
		)
	)

	differences_3 = dict([difference for sublist in differences_3 for difference in sublist])

pd.DataFrame(get_sorted_dict(differences_3)[:10], columns = ['Initial State', 'Hemming Distance'])


g_0 = x2 + x3

T = 812
----------
CPU times: user 1.13 s, sys: 186 ms, total: 1.32 s
Wall time: 1min 58s


,Initial State,Hemming Distance
0,"(305, 842)",318
1,"(498, 611)",339
2,"(1021, 304)",339
3,"(66, 733)",342
4,"(24, 779)",343
5,"(340, 883)",343
6,"(817, 108)",343
7,"(129, 513)",344
8,"(225, 816)",344
9,"(446, 966)",345


In [53]:
poly_state_2 = 305
poly_state_3 = 842

In [54]:
l = 50000

sequence_poly_0 = generate_sequence(polynomials_int[0], poly_state_0, l)
sequence_poly_1 = generate_sequence(polynomials_int[1], poly_state_1, l)
sequence_poly_2 = generate_sequence(polynomials_int[2], poly_state_2, l)
sequence_poly_3 = generate_sequence(polynomials_int[3], poly_state_3, l)
sequence_poly_4 = generate_sequence(polynomials_int[4], poly_state_4, l)
sequence_poly_5 = generate_sequence(polynomials_int[5], poly_state_5, l)

sequence = [
	int(boolean_combining_function(
		state_to_int((0, sequence_poly_0[i]), (1, sequence_poly_1[i]), (2, sequence_poly_2[i]), (3, sequence_poly_3[i]), (4, sequence_poly_4[i]), (5, sequence_poly_5[i]))
	))
	for i in range(l)
]

print(sequence[:15])
print(validated_gamma[:15])

get_bit_difference(sequence, validated_gamma[:l])


[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]


0

In [55]:
int(''. join ([ str (i) for i in [0, 0, 1] ]) , 2)

1